# 乗降客数

In [ ]:
stations = pd.read_csv('input/stations_tokai3ken_edited.csv', index_col=0)

In [ ]:
stations

In [ ]:
#sns.pointplot(x="乗降客数（日）", y="飲食店事業所数", hue="路線ID", data=stations)

In [ ]:
stations['路線ID'].value_counts()

In [ ]:
a_23 = stations[stations['路線ID'] == 'a_23.0'].sort_values(by='乗降客数（日）')
m_2 = stations[stations['路線ID'] == 'm_2.0'].sort_values(by='乗降客数（日）')
g_0 = stations[stations['路線ID'] == 'g_0.0'].sort_values(by='乗降客数（日）')
m_12 = stations[stations['路線ID'] == 'm_12.0'].sort_values(by='乗降客数（日）')
a_3 = stations[stations['路線ID'] == 'a_3.0'].sort_values(by='乗降客数（日）')
a_0 = stations[stations['路線ID'] == 'a_0.0'].sort_values(by='乗降客数（日）')
#rosen_vc = pd.concat([a_23, m_2, g_0, m_12, a_3, a_0], axis=0)
#rosen_vc_log = rosen_vc.copy()
#rosen_vc_log['飲食店事業所数'] = np.log1p(rosen_vc['飲食店事業所数'].values)

In [ ]:
#print(a_23['乗降客数（日）'].max(), a_0['乗降客数（日）'].max(), a_3['乗降客数（日）'].max())

In [ ]:
#plt.figure(figsize=(16, 12))
#sns.pointplot(x="乗降客数（日）", y="飲食店事業所数", hue="路線ID", data=rosen_vc)

In [ ]:
#plt.figure(figsize=(16, 12))
#sns.pointplot(x="乗降客数（日）", y="飲食店事業所数", hue="路線ID", data=rosen_vc_log)

In [ ]:
#plt.figure(figsize=(16, 12))
#sns.pointplot(x="乗降客数（日）", y="飲食店事業所数", data=a_0, color='black')
#sns.pointplot(x="乗降客数（日）", y="飲食店事業所数", data=a_3, color='yellow')
#sns.pointplot(x="乗降客数（日）", y="飲食店事業所数", data=m_12, color='purple')
#sns.pointplot(x="乗降客数（日）", y="飲食店事業所数", data=g_0, color='green')
#sns.pointplot(x="乗降客数（日）", y="飲食店事業所数", data=m_2, color='blue')
#sns.pointplot(x="乗降客数（日）", y="飲食店事業所数", data=a_23, color='red')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(a_23["乗降客数（日）"], a_23["飲食店事業所数"], 'o-')
plt.plot(m_2["乗降客数（日）"], m_2["飲食店事業所数"], 'o-')
plt.plot(g_0["乗降客数（日）"], g_0["飲食店事業所数"], 'o-')
plt.plot(m_12["乗降客数（日）"], m_12["飲食店事業所数"], 'o-')
plt.plot(a_3["乗降客数（日）"], a_3["飲食店事業所数"], 'o-')
plt.plot(a_0["乗降客数（日）"], a_0["飲食店事業所数"], 'o-')

# ここから

In [ ]:
import pystan

In [ ]:
X = pd.concat([stations['乗降客数（日）'], stations['路線ID']], axis=1)
y = stations['飲食店事業所数']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
#多治見、土岐、(瑞浪、)恵那、中津川をtestに移動
X_tono = pd.DataFrame(data=[X_train.loc['g_1189'], X_train.loc['g_1190'], X_train.loc['g_1194'], X_train.loc['g_1196']])
X_test = pd.concat([X_test, X_tono], axis=0)
X_train = X_train.drop(['g_1189', 'g_1190', 'g_1194', 'g_1196'], axis=0)
y_tono = pd.Series(data=[y_train.loc['g_1189'], y_train.loc['g_1190'], y_train.loc['g_1194'], y_train.loc['g_1196']],
                      index=['g_1189', 'g_1190', 'g_1194', 'g_1196'])
y_test = pd.concat([y_test, y_tono], axis=0)
y_train = y_train.drop(['g_1189', 'g_1190', 'g_1194', 'g_1196'], axis=0)

In [ ]:
#X_train

In [ ]:
rosen = dict(zip(stations['路線ID'].unique(), range(1, 1+len(stations['路線ID'].unique()))))
Rosen_train = []
Rosen_test = []
for i in range(len(X_train)):
    Rosen_train.append(rosen[X_train['路線ID'][i]])
for i in range(len(X_test)):
    Rosen_test.append(rosen[X_test['路線ID'][i]])

data = dict(
    N_train=X_train.shape[0],
    N_test=X_test.shape[0],
    N_rosen=len(rosen),
    Joko_train=X_train['乗降客数（日）'],
    Joko_test=X_test['乗降客数（日）'],
    Rosen_train=Rosen_train,
    Rosen_test=Rosen_test,
    Inshoku_train=y_train.values
)

In [ ]:
model = '''
data {
    int N_train;
    int N_test;
    int N_rosen;
    vector<lower=0>[N_train] Joko_train;
    vector<lower=0>[N_test] Joko_test;
    int<lower=1> Rosen_train[N_train];
    int<lower=1> Rosen_test[N_test];
    vector<lower=0>[N_train] Inshoku_train;
}

parameters {
    real mu_a;
    real mu_b;
    real<lower=0> mu_s;
    real<lower=0> s_a;
    real<lower=0> s_b;
    real<lower=0> s_s;
    vector[N_rosen] a;
    vector[N_rosen] b;
    vector[N_rosen] s;
}

model {
    a ~ normal(mu_a, s_a)
    b ~ normal(mu_b, s_b)
    s ~ normal(mu_s, s_s)
    for (i in 1:N_train)
        Inshoku_train[i] ~ normal(a[Rosen_train[i]] + b[Rosen_train[i]]*Joko_train, s[Rosen_train[i]]);
}

generated quantities{
    vector[N_test] predict;
    for (i in 1:N_test)
        predict[i] = normal_rng(a[Rosen_test[i]] + b[Rosen_test[i]]*Joko_test[i], s[Rosen_test[i]]);
}
'''

In [ ]:
fit = pystan.stan(model_code=model, data=data, chains=3, iter=1000, warmup=200, thin=1)

In [ ]:
fit

In [ ]:
fit.extract()['predict'][0]

In [ ]:
#y_test

In [ ]:
predict = fit.extract()['predict'][0]

In [ ]:
fig, ax = plt.subplots()
ax.scatter(predict, y_test, edgecolors=(0, 0, 0))
ax.plot([predict.min(), predict.max()], [predict.min(), predict.max()], 'k--', lw=1)
#ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=1)
ax.set_xlabel('Predicted')
ax.set_ylabel('Measured')
plt.show()